In [1]:
# Importing Modules

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup
import os
import pdfkit

In [3]:
# Getting All site Information

http = urllib3.PoolManager()
url = "https://www.javatpoint.com"
r = http.request('GET', url)
soup = BeautifulSoup(r.data, 'html.parser')

In [4]:
allContent = soup.findAll('fieldset', attrs={'class': 'gra1'})

In [14]:
# Manipulating Site Information 

fieldNames = []
topics = []
for field in allContent:
    headline = field.find('h2').text
    fieldNames.append(headline)
    contentNames = field.findAll('p')
    contentLinks = field.findAll('a')
    fields = []
    for i in range(len(contentNames)):
        if(contentLinks[i]['href'][0:4]!='http'):
            contentLinks[i]['href'] = 'http://www.javatpoint.com/' + contentLinks[i]['href']
        fields.append((contentNames[i].text, contentLinks[i]['href']))
    topics.append(fields)

In [48]:
# Define User choice

def getIndex(fieldNames):
    try:
        ind = int(input('Select one index: '))
        if(ind>0 and ind<=len(fieldNames)):
            return ind
        else:
            print('Please be a human!')
            return getIndex(fieldNames)
    except:
        print('Please Select a number')
        return getIndex(fieldNames)

In [50]:
# All Topics

for i in range(len(fieldNames)):
    print('{0}. {1}'.format(i+1, fieldNames[i]))
    for x in topics[i]:
        print('\t{0}'.format(x[0]))

1. Latest Tutorials
	Swift
	Pig
	Flask
	C. Graphics
	Automata
	Testing
2. B.Tech / MCA
	DBMS
	DS
	DAA
	OS
	C. Network
	Compiler D.
	COA
	D. Math.
	E. Hacking
	Web Tech.
	Cyber Sec.
	C
	C++
	Java
	.Net
	Python
	Programs
	Control S.
3. Popular Tutorials
	DS
	Java
	C#
	C
	C++
	SQL
	Android
	Python
	Cloud
	Hadoop
	Excel
	Computer
	Linux
	vi editor
	Embedded
	Robotics
	SEO
	F#
	Powerpoint
	Shell Script
	MathML
	Perl
	Scala
	MS Word
	Ruby
	Rails
	Docker
	ASP.NET
	ADO.NET
	GWT
	Go Lang
	On Ubuntu
	Phalcon
	On Mac
	On CentOS
	SVG
	Polymer
	Compiler D.
	Django
	.Net
	OS
	Kotlin
	Aptitude
	Rust
	Control S.
	Selenium
	IoT
	Tkinter
	TypeScript
	React Native
	AWS
	NumPy
4. Database Tutorials
	SQL
	PL/SQL
	Oracle
	MySQL
	PostgreSQL
	MongoDB
	Cassandra
	SQLite
	Neo4j
	CouchDB
	SQL Server
	MariaDB
	DB2
	PouchDB
	Redis
5. Web Technology
	HTML
	CSS
	JavaScript
	jQuery
	jQuery UI
	PHP
	XML
	JSON
	AJAX
	Web Serv-
	Bootstrap
	AngularJS
	Node.js
	Express.js
	Backbone
	Sass
	Less
	WordPress
	CodeIgniter
	XSL

In [62]:
# Get User choice

ind1 = getIndex(fieldNames)
for i in range(len(topics[ind1-1])):
    print('{0}. {1}'.format(i+1, topics[ind1-1][i][0]))
ind2 = getIndex(topics[ind1-1])

topicNameDecided = topics[ind1-1][ind2-1][0]
topicLinkDecided = topics[ind1-1][ind2-1][1]
print(topics[ind1-1][ind2-1][0], topics[ind1-1][ind2-1][1])

Select one index: 3
1. DS
2. Java
3. C#
4. C
5. C++
6. SQL
7. Android
8. Python
9. Cloud
10. Hadoop
11. Excel
12. Computer
13. Linux
14. vi editor
15. Embedded
16. Robotics
17. SEO
18. F#
19. Powerpoint
20. Shell Script
21. MathML
22. Perl
23. Scala
24. MS Word
25. Ruby
26. Rails
27. Docker
28. ASP.NET
29. ADO.NET
30. GWT
31. Go Lang
32. On Ubuntu
33. Phalcon
34. On Mac
35. On CentOS
36. SVG
37. Polymer
38. Compiler D.
39. Django
40. .Net
41. OS
42. Kotlin
43. Aptitude
44. Rust
45. Control S.
46. Selenium
47. IoT
48. Tkinter
49. TypeScript
50. React Native
51. AWS
52. NumPy
Select one index: 20
Shell Script http://www.javatpoint.com/shell-scripting-tutorial


In [63]:
# Getting Modules

http = urllib3.PoolManager()
url = topicLinkDecided
r = http.request('GET', url)
soup = BeautifulSoup(r.data, 'html.parser')

In [64]:
# Creating Modules

moduleTags = soup.findAll('h2', attrs={'class': 'spanh2'})
modules = []
for module in moduleTags:
    modules.append(module.text)

for module in modules:
    if not os.path.exists(module):
        os.makedirs(module)

In [65]:
# Creating HTML files from each modules

reqNo=0
contentTable = soup.findAll('div', attrs={'class': 'leftmenu'}) # Index

for i in range(len(contentTable)):
    links = []
    for x in contentTable[i]:
        a = str(x)
        y = "href="
        i1 = a.find(y)
        if(i1==-1): # No Links
            continue
        i1+=6 # Skip <"a href=">
        i2 = a.find('\"', i1)  # Find Ending quote
        links.append(a[i1:i2]) # a[i1:i2] is the final href
    print(links)
    for link in links:
        reqNo+=1
        print("Request Number:" + str(reqNo))
        url = "http://www.javatpoint.com/" + link
        r = http.request('GET', url)
        soup = BeautifulSoup(r.data, 'html.parser')
        [s.extract() for s in soup('script')]  # Remove Scripts
        [s.extract() for s in soup('ins')]     # Remove Ads
        [s.extract() for s in soup('iframe')]  # Remove Ads2
        mainContent = soup.find('div', attrs={'id': 'city'})

        # For Images
        images = mainContent.findAll('img')
        for x in range(len(images)):
            imageSrc = images[x]['src']
            imageSrc = str(imageSrc)
            if(imageSrc.find('static.javatpoint.com')==-1):
                images[x]['src'] = 'http://javatpoint.com' + images[x]['src']

        for x in range(len(images)):
            mainContent.findAll('img')[x]['src']=images[x]['src']
            
        # For CodeBlock
        codes = mainContent.findAll('textarea', attrs= {'name': 'code'})
        for c in range(len(codes)):
            newTag = soup.new_tag('p')
            newTag.append(str(codes[c].get_text()))
            mainContent.find('textarea').replaceWith(newTag)

        # Final HTML
        mainContent = str(mainContent).replace("\n"," ")
        header = "<html> <head> <meta charset='UTF-8'> <link rel=\"stylesheet\" type=\"text/css\" href=\"./../javatpoint.css\"> </head>  <body>"
        footer = "</body></html>"
        mainContent = header + mainContent + footer
        try:
            f = open("./" + modules[i] + "/" + link + ".html", "w")
            for ch in mainContent:
                try:
                    f.write(ch)
                except:
                    f.write(" ")
            f.close()
        except:
            print("Can't Create " + str(link) + ".html")


['shell-scripting-tutorial', 'what-is-shell-scripting', 'how-to-determine-shell-scripting', 'shell-scripting-she-bang', 'shell-scripting-comments', 'shell-scripting-variables', 'shell-scripting-sourcing-a-file', 'troubleshooting-a-shell-script', 'shell-scripting-prevent-setuid-root-spoofing']
Request Number:1
Request Number:2
Request Number:3
Request Number:4
Request Number:5
Request Number:6
Request Number:7
Request Number:8
Request Number:9
['steps-to-write-and-execute-a-shell-script']
Request Number:10
['shell-script-parameters', 'shell-scripting-shift-through-parameters']
Request Number:11
Request Number:12
['shell-scripting-sourcing-a-config-file']
Request Number:13
['get-shell-script-options-with-getopts']
Request Number:14
['if-then-else-shell-scripting', 'if-then-elif-shell-scripting', 'for-loop-shell-scripting', 'while-loop-shell-scripting', 'until-loop-shell-scripting']
Request Number:15
Request Number:16
Request Number:17
Request Number:18
Request Number:19
['scripting-shell

In [66]:
# Sorting HTML according to creation time

def sorted_dir(folder):
    def getctime(name):
        path = os.path.join(folder, name)
        return os.path.getctime(path)
    return sorted(os.listdir(folder), key=getctime)

# For Windows only
# path_wkthmltopdf = "C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe"
# config = pdfkit.configuration(wkhtmltopdf=path_wkthmltopdf)


In [69]:
# Converting each module to PDF

for x in range(len(modules)):
    myList = sorted_dir(modules[x])
    for i in range(len(myList)):
        myList[i]=modules[x] + "/" + myList[i]
    pdfkit.from_file(myList, modules[x] + '.pdf')

Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                        
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                       

In [70]:
modules

['Shell Scripting',
 'Executing Script',
 'Shell Parameters',
 'Shell Sourcing',
 'Shell Getopts',
 'Shell Loops',
 'Advance Shell']